In [2]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

%load_ext autoreload
%autoreload 2

import transformers
transformers.set_seed(17)


2024-06-14 13:38:41.581114: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# What should be the max length of the model?

Approach: Tokenize the longest annotation in the dataset, this is the max lenght.

In [ ]:
from src.dataset import TDMSDataset, PATH


# load all annotations:
dataset = TDMSDataset(PATH.TRAIN)

annotations = []
for i in range(len(dataset)):
    f, tex, jsn = dataset[i]
    annotations.append(str(jsn))


In [ ]:
import pandas as pd


df = pd.DataFrame(annotations)
df.columns = ["annotation"]
df

In [ ]:
df["str_len"] = df.apply(lambda row: len(row["annotation"]), axis=1)

# top 10 longest annotations by string lenght:
df_t10 = df.sort_values(by="str_len", ascending=False)[:10]
df_t10

In [ ]:
# ignoring the outlier of 56561 chars and all below 20k chars as candidates for longest annotation

df_longest = df_t10[1:6]
df_longest


In [ ]:
# is the longest annotation character wise also the longest annotation token wise?
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 9878 tokens => Max len should be 10k tokens

In [ ]:
# Sanity check: Other Tokenizers (llama)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 8670 tokens => Max len should be 8700 tokens

In [ ]:
# Sanity check: Other Tokenizers (gemma)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 8898 tokens => Max len should be 9k tokens

# Model Experiments

In [1]:
from src.experiment_runner import Experiment, run
from src.dataset import PATH
from src.prompt_templates import simple_zs,simple_fs_v2, simple_fs_v3,simple_fs_v4a, simple_fs_v4b, simple_fs_v5, simple_zs_v5, simple_fs, few_shot_template_initial
from src.content_extraction import naive_doctaet, format
from src.models import Model, OllamaModel, ARCHITECTURE
import torch
from itertools import product
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)


all_models = [
    ARCHITECTURE.MISTRAL_7b,
    ARCHITECTURE.CODESTRAL,
    ARCHITECTURE.MISTRAL_7b_IT,
    ARCHITECTURE.GEMMA_7b,
    ARCHITECTURE.GEMMA_2b,
    ARCHITECTURE.LLAMA_8b,
    ARCHITECTURE.CMD_RPLUS,
    ARCHITECTURE.LLAMA_8b_IT,
    ARCHITECTURE.LLAMA_70b,
    ARCHITECTURE.MIXTRAL_22B,
    ARCHITECTURE.MIXTRAL_7B,
    ARCHITECTURE.MIXTRAL_7B_IT,
    ARCHITECTURE.MIXTRAL_22B_IT,
]

smallish_models = [
    ARCHITECTURE.MISTRAL_7b,
    ARCHITECTURE.MISTRAL_7b_IT,
    ARCHITECTURE.GEMMA_7b,
    ARCHITECTURE.GEMMA_2b,
    ARCHITECTURE.LLAMA_8b,
    ARCHITECTURE.LLAMA_8b_IT,
    ARCHITECTURE.MIXTRAL_7B,
    ARCHITECTURE.MIXTRAL_7B_IT,
]

larger_models = [
    ARCHITECTURE.CODESTRAL,
    ARCHITECTURE.CMD_RPLUS,
    ARCHITECTURE.LLAMA_70b,
    ARCHITECTURE.MIXTRAL_22B,
    ARCHITECTURE.MIXTRAL_22B_IT,
]

instruct_models = [
    ARCHITECTURE.MISTRAL_7b_IT,
    ARCHITECTURE.LLAMA_8b_IT,
    ARCHITECTURE.MIXTRAL_7B_IT,
    ARCHITECTURE.MIXTRAL_22B_IT,
]

models = smallish_models

prompts = [
    # few_shot_template_initial,
    simple_fs_v5,
    simple_zs_v5,
]

# Summarization & Postprocessing are static
exps = [Experiment(model, prompt, naive_doctaet) for model, prompt in product(models, prompts)]
print(f"running {len(exps)} experiments")


for exp in exps:
    try:
        _ = run(exp, PATH.VAL, post_proc=format)
    except:
        print(f"Experiment {exp} failed")  


ModuleNotFoundError: No module named 'src'

In [7]:
from src.models import ARCHITECTURE
import ollama

for k, arch in ARCHITECTURE.__dict__.items():
    try:
        print(k)
        res = ollama.pull(arch.ollama)
        print(res)
    except Exception as e:
        print(f"{k}: {e}")

__module__
__module__: 'str' object has no attribute 'ollama'
MISTRAL_7b
{'status': 'success'}
CODESTRAL
{'status': 'success'}
MISTRAL_7b_IT
{'status': 'success'}
GEMMA_7b
{'status': 'success'}
GEMMA_2b
{'status': 'success'}
LLAMA_8b
{'status': 'success'}
CMD_RPLUS


In [ ]:
ARCHITECTURE.__id